# Leverage calculations from market data

A simple notebook showing leverage calculations based on market data obtined via the API.

Resources:
- [Testnet](https://console.fairground.wtf/)
- [API docs](https://docs.fairground.vega.xyz/docs/apis/)

## [gRPC](https://docs.fairground.vega.xyz/docs/apis/grpc/)

In [1]:
pip install --upgrade Vega-API-client;

Note: you may need to restart the kernel to use updated packages.


In [2]:
import vegaapiclient as vac
import math
node_url_grpc="n06.testnet.vega.xyz:3002"
data_client = vac.VegaTradingDataClient(node_url_grpc)

def round_sd(number):
    significant_digits = 4
    return round(number, significant_digits - math.floor(math.log10(abs(number))) - 1)

### Specify public key for the party

Specify the your public key in the cell below. It can be found in the [wallet side panel](https://console.fairground.wtf/wallet/) in the Console.

![](./resources/pubkey.png)

In [3]:
pubkey="e23f51fa4236ac4c843b56fed310db713055dae5d5ce71b976465295206b11bf"

In [4]:
party_positions = data_client.PositionsByParty(vac.api.trading.PositionsByPartyRequest(party_id=pubkey))
if len(party_positions.positions) == 0:
    print("party with the public key specified has no open positions")
    exit()

party_accounts = data_client.PartyAccounts(vac.api.trading.PartyAccountsRequest(party_id=pubkey))

for pos in party_positions.positions:
    market_name=data_client.MarketByID(vac.api.trading.MarketByIDRequest(market_id=pos.market_id)).market.tradable_instrument.instrument.name
    market_data=data_client.MarketDataByID(vac.api.trading.MarketDataByIDRequest(market_id=pos.market_id)).market_data
    margin_acc = next(x for x in party_accounts.accounts if x.market_id == pos.market_id )
    gen_acc = next(x for x in party_accounts.accounts if x.type == vac.vega.ACCOUNT_TYPE_GENERAL and x.asset==margin_acc.asset)
    
    notional=pos.open_volume*market_data.mark_price
    current_leverage=round_sd(notional/(gen_acc.balance+margin_acc.balance))


    margin_levels=data_client.MarginLevels(vac.api.trading.MarginLevelsRequest(party_id=pubkey, market_id=pos.market_id)).margin_levels

    print("\tmarket: {name}".format(name=market_name))
    print("\t\teffective position leverage: ~{lev}x".format(lev=current_leverage))
    print("\t\tleverage per margin level:")
    print("\t\t\t{:<20} ~{}x".format("maintenance:",round_sd(notional/margin_levels[0].maintenance_margin)))
    print("\t\t\t{:<20} ~{}x".format("search",round_sd(notional/margin_levels[0].search_level)))
    print("\t\t\t{:<20} ~{}x".format("initial",round_sd(notional/margin_levels[0].initial_margin)))
    print("\t\t\t{:<20} ~{}x".format("release",round_sd(notional/margin_levels[0].collateral_release_level)))



	market: Apple Monthly (30 Jun 2021)
		effective position leverage: ~0.1292x
		leverage per margin level:
			maintenance:         ~117.5x
			search               ~114.7x
			initial              ~111.9x
			release              ~106.8x


## [GraphQL](https://docs.fairground.vega.xyz/docs/apis/graphql/)

All the data required for above calculations can also be obtained via the GraphQL API. Try going to the [GraphQL playground](https://lb.testnet.vega.xyz/playground) for the testnet and entering the query below (remember to modify the public key to yours) 

```bash
query($pubKey: ID = "e23f51fa4236ac4c843b56fed310db713055dae5d5ce71b976465295206b11bf") {        
  party(id:$pubKey){
    positions {
      market {
        name,
      	data {
          markPrice
        }}
      openVolume,
      margins {
        maintenanceLevel
       	searchLevel,
        initialLevel,
        collateralReleaseLevel,
      }
    },
    accounts {
      type,
      balance,
      asset {
        name
      }, 
      market {
        name
      }
    }
  }
}
```